# 非同期推論実行方法の基礎を学ぶ
OpenVINOを使って効率の良い推論プログラムを作るには非同期推論(asynchronous inference)を活用するのが重要です。  
同期推論(synchronous inference)で使用する`infer()` APIはブロッキング関数なので、プログラムは推論が終わるまで待たされてしまうので他の処理を同時に行うことができません。

### 入力画像とラベルデータの準備
まずは推論に使用する入力画像ファイルと、クラスラベルのテキストファイルをOpenVINOのdemoディレクトリからコピーしてきます。

In [ ]:
# Linux
!cp $INTEL_OPENVINO_DIR/deployment_tools/demo/car.png .
!cp $INTEL_OPENVINO_DIR/deployment_tools/demo/squeezenet1.1.labels synset_words.txt

In [ ]:
# Windows
!copy "%INTEL_OPENVINO_DIR%\deployment_tools\demo\car.png" .
!copy "%INTEL_OPENVINO_DIR%\deployment_tools\demo\squeezenet1.1.labels" synset_words.txt

コピーしてきた推論入力の絵を表示して確認します。

In [ ]:
from IPython.display import Image
Image('car.png')

### 推論に使用するIRモデルデータの準備
推論に使用するモデルを`Model downloader`でダウンロードし、`Model converter`でIRモデルに変換します。

In [ ]:
# Linux
!python3 $INTEL_OPENVINO_DIR/deployment_tools/tools/model_downloader/downloader.py --name googlenet-v1
!python3 $INTEL_OPENVINO_DIR/deployment_tools/tools/model_downloader/converter.py  --name googlenet-v1 --precisions FP16
!ls public/googlenet-v1/FP16 -l

In [ ]:
# Windows
!python "%INTEL_OPENVINO_DIR%\deployment_tools\tools\model_downloader\downloader.py" --name googlenet-v1
!python "%INTEL_OPENVINO_DIR%\deployment_tools\tools\model_downloader\converter.py"  --name googlenet-v1 --precisions FP16
!dir public\googlenet-v1\FP16

----
ここからPythonプログラム本体となります。  

### OpenVINOアプリケーションの初期化を行う
この部分は前の画像分類プログラムと全く同じです。
1. 必要なPythonモジュールをインポートする
2. クラスラベルデータを読み込む
3. Inference engineのcoreオフジェクトの生成
4. IRモデルをメモリに読み込む
5. Input / Outputブロブの情報の取得
6. モデルをIE coreオブジェクトにロード

In [ ]:
import time

import cv2
import numpy as np
from openvino.inference_engine import IENetwork, IECore

label = open('synset_words.txt').readlines()

# Inference Engineコアオブジェクトの生成
ie = IECore()

# IRモデルファイルの読み込み
model = './public/googlenet-v1/FP16/googlenet-v1'
net = IENetwork(model=model+'.xml', weights=model+'.bin')

# 入出力blobの名前の取得、入力blobのシェイプの取得
input_blob_name  = list(net.inputs.keys())[0]
output_blob_name = list(net.outputs.keys())[0]
batch,channel,height,width = net.inputs[input_blob_name].shape

exec_net = ie.load_network(network=net, device_name='CPU', num_requests=1)

### コールバック関数をセットする
コールバック関数にラムダ式(無名関数)を指定することも可能です。 
`set_completion_callback()`関数の第２引数はユーザーデータで、コールバック関数に渡されます。ここでは`exec_net.requests[req_id]`を渡すことで、コールバック関数内で結果の表示ができるようにしています。
コールバック関数の中では推論結果の表示をさせています。

In [ ]:
def callback(status_code, infer_request):
    output = infer_request.outputs[output_blob_name][0]
    idx = np.argsort(output)[::-1]
    for i in range(5):
        print(idx[i]+1, output[idx[i]], label[idx[i]][:-1])

req_id=0
exec_net.requests[req_id].set_completion_callback(callback, exec_net.requests[req_id])

### 推論入力データを準備する
推論入力画像を読み込み、入力ブロブのシェイプに合わせて変形します。

In [ ]:
img = cv2.imread('car.png')
img = cv2.resize(img, (width,height))
img = img.transpose((2, 0, 1))
img = img.reshape((1, channel, height, width))

### **非同期**推論を実行する  
`start_async()` APIを実行すると実際の推論は別スレッドで実行され、すぐに制御が戻ってきます。 そのため、数字を表示する処理が開始されますが、その途中で推論が終わり、コールバック関数が呼ばれて推論結果が表示されるのがわかると思います。

In [ ]:
res = exec_net.start_async(0, inputs={input_blob_name: img})

for i in range(10):
    print(i)
    time.sleep(0.1)    # 0.1秒待つ

----
## ここまでで非同期推論プログラムの作り方の基礎について学びました
非同期推論を活用することでハードウエア資源を効率よく利用することが可能になります。  
内蔵GPU, VPU (Myriad)などの推論デバイスを使用する際も、せっかく処理をCPUからオフロードしたのに推論終了までCPUの処理が停止していては意味がありません。  
非同期推論の使い方をぜひマスターして、効率よいOpenVINOアプリケーション作成を行ってください。

## Next => 非同期推論、同時推論を使った高スループットアプリケーションの基礎 - [classification-async-multi.ipynb](./classification-async-multi.ipynb)